In [4]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [6]:
subject = 'Goiás - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [7]:
data = pd.read_csv('2003_01_model_input_GO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Goiás - Desemprego,Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - Consumo de Cimento (t)
0,2003,8.179785,0.745293,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,...,1448.938253,23.439039,15.642757,66.094986,1.445646,9.371822e+07,5.518325e+06,14.031270,8.418248e+07,1311.935000
1,2004,8.111394,0.746636,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,...,1396.777198,23.790752,16.201501,66.339648,1.399708,9.513036e+07,5.577901e+06,14.069740,8.464257e+07,1514.939000
2,2005,8.088033,0.747851,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,...,1342.159073,24.037182,16.563211,66.178990,1.315326,9.643551e+07,5.616938e+06,14.103822,8.501827e+07,1716.900000
3,2006,8.002287,0.748941,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,...,1330.689066,23.844192,16.110769,65.564132,1.286416,9.771534e+07,5.664472e+06,14.152999,8.544612e+07,1730.142000
4,2007,7.971612,0.749908,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,...,1346.447917,23.589271,15.814321,65.951670,1.274907,9.885878e+07,5.710316e+06,14.189755,8.577093e+07,1943.769000
5,2008,7.941146,0.750753,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,...,1376.621797,23.533988,16.021792,67.029130,1.258587,9.979485e+07,5.727076e+06,14.203138,8.592012e+07,2310.616000
6,2009,7.969157,0.751479,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,...,1404.486199,23.539793,15.895700,66.852293,1.302609,1.005138e+08,5.757645e+06,14.197250,8.596210e+07,2335.717000
7,2010,7.932921,0.752088,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,...,1422.397943,23.407790,15.594180,66.528157,1.346857,1.030813e+08,5.806966e+06,14.241313,8.663480e+07,2657.527000
8,2011,7.921619,0.752581,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,...,1424.732773,23.379311,15.713953,67.017525,1.343835,1.054535e+08,5.798734e+06,14.278825,8.718791e+07,2901.707000
9,2012,7.786290,0.752723,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,...,1426.391027,23.445549,15.826075,67.022758,1.300550,1.075784e+08,5.775231e+06,14.289292,8.762067e+07,3061.533000


In [8]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Goiás - Desemprego,Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiás - Produção de Cimento (t),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado
0,8.179785,0.745293,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,707.997208,...,932.625368,1448.938253,23.439039,15.642757,66.094986,1.445646,9.371822e+07,5.518325e+06,14.031270,8.418248e+07
1,8.111394,0.746636,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,747.779683,...,932.555747,1396.777198,23.790752,16.201501,66.339648,1.399708,9.513036e+07,5.577901e+06,14.069740,8.464257e+07
2,8.088033,0.747851,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,773.277269,...,932.501416,1342.159073,24.037182,16.563211,66.178990,1.315326,9.643551e+07,5.616938e+06,14.103822,8.501827e+07
3,8.002287,0.748941,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,813.650774,...,932.061007,1330.689066,23.844192,16.110769,65.564132,1.286416,9.771534e+07,5.664472e+06,14.152999,8.544612e+07
4,7.971612,0.749908,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,924.314058,...,931.363036,1346.447917,23.589271,15.814321,65.951670,1.274907,9.885878e+07,5.710316e+06,14.189755,8.577093e+07
5,7.941146,0.750753,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,1049.218132,...,931.224932,1376.621797,23.533988,16.021792,67.029130,1.258587,9.979485e+07,5.727076e+06,14.203138,8.592012e+07
6,7.969157,0.751479,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,1167.838500,...,931.509364,1404.486199,23.539793,15.895700,66.852293,1.302609,1.005138e+08,5.757645e+06,14.197250,8.596210e+07
7,7.932921,0.752088,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,1306.390438,...,931.774922,1422.397943,23.407790,15.594180,66.528157,1.346857,1.030813e+08,5.806966e+06,14.241313,8.663480e+07
8,7.921619,0.752581,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,1430.447037,...,932.026640,1424.732773,23.379311,15.713953,67.017525,1.343835,1.054535e+08,5.798734e+06,14.278825,8.718791e+07
9,7.786290,0.752723,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,1500.429389,...,932.153867,1426.391027,23.445549,15.826075,67.022758,1.300550,1.075784e+08,5.775231e+06,14.289292,8.762067e+07


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     1514.939000
1     1716.900000
2     1730.142000
3     1943.769000
4     2310.616000
5     2335.717000
6     2657.527000
7     2901.707000
8     3061.533000
9     3181.778000
10    2861.897429
11    2517.290743
12    2193.545829
13    2270.346000
14    2282.903000
15    2367.716000
16    2438.585000
17    2782.135000
18    2807.973000
19            NaN
Name: Goiás - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Goiás - Desemprego,Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiás - Produção de Cimento (t),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado
0,-0.419401,-2.134050,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-1.556914,...,1.014512,1.265709,-1.211646,-0.201870,0.426873,1.987057,-1.653672,-1.577371,-2.271887,-1.973835
1,-0.545138,-1.684552,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-1.444254,...,0.855972,-0.259411,0.323498,1.290383,0.597823,1.456691,-1.438444,-0.939271,-1.704457,-1.604809
2,-0.588088,-1.277751,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-1.372048,...,0.732250,-1.856372,1.399107,2.256416,0.485569,0.482488,-1.239523,-0.521162,-1.201734,-1.303461
3,-0.745733,-0.912947,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-1.257714,...,-0.270643,-2.191739,0.556753,1.048065,0.055956,0.148713,-1.044463,-0.012043,-0.476356,-0.960295
4,-0.802129,-0.589437,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.944328,...,-1.860054,-1.730972,-0.555916,0.256331,0.326736,0.015841,-0.870187,0.478975,0.065791,-0.699769
5,-0.858140,-0.306523,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.590614,...,-2.174542,-0.848728,-0.797216,0.810430,1.079577,-0.172583,-0.727519,0.658485,0.263200,-0.580109
6,-0.806643,-0.063503,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.254694,...,-1.526837,-0.034010,-0.771877,0.473672,0.956018,0.335668,-0.617937,0.985892,0.176354,-0.546440
7,-0.873262,0.140325,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,0.137669,...,-0.922113,0.489705,-1.348041,-0.331608,0.729538,0.846515,-0.226624,1.514151,0.826284,-0.006881
8,-0.894041,0.305660,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,0.488984,...,-0.348903,0.557973,-1.472345,-0.011727,1.071469,0.811624,0.134936,1.425985,1.379601,0.436759
9,-1.142844,0.353208,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.687166,...,-0.059185,0.606458,-1.183231,0.287721,1.075125,0.311887,0.458795,1.174255,1.533993,0.783868


In [11]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     1514.939000
1     1716.900000
2     1730.142000
3     1943.769000
4     2310.616000
5     2335.717000
6     2657.527000
7     2901.707000
8     3061.533000
9     3181.778000
10    2861.897429
11    2517.290743
12    2193.545829
13    2270.346000
14    2282.903000
15    2367.716000
16    2438.585000
Name: Goiás - Consumo de Cimento (t), dtype: float64

In [12]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Goiás - Desemprego,Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiás - Produção de Cimento (t),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado
17,0.596674,-0.104074,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.913043,...,0.207996,0.427875,0.763149,-1.965098,-1.88718,-2.218598,0.832746,-0.850806,-0.449768,0.60687


In [13]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    2782.135
Name: Goiás - Consumo de Cimento (t), dtype: float64

In [14]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [15]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [17]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[188823716, 21718807, 2897284559, 122553557, 68639920, 1627279305, 134012554, 2656371677, 548836201, 3485523134, 4023811922, 3780404887, 2205848068, 168927741, 831600968, 3196649059, 2254914922, 2892248854, 1738785081, 131454448, 3534256534, 721839714, 3296427990, 2453527369, 1199324954, 2822395646, 711320232, 2053918259, 3477860538, 1025353003, 2507137264, 2448108227, 1482039281, 3936010801, 1781017298, 1151885781, 352752931, 4090696969, 978440652, 1094002451, 2820030365, 2757215445, 4058937905, 1747373622, 855446939, 2516363368, 3839483784, 3870022776, 1276572866, 2905878807, 791528673, 1227478328, 307132959, 3122167227, 3829640723, 1058012047, 4111899634, 3015295116, 2965731392, 607109512, 2007452138, 3409438305, 1756660324, 3946441482, 2729572846, 2579552431, 2561944010, 3222293743, 2973931782, 3440740309, 2561431289, 391376743, 2434241278, 38390148, 2206977546, 3043235159, 3391622690, 2645304086, 1321471324, 2740503371, 1498524162, 3724748127, 3051423063, 2290603962, 3369358098, 2

val_loss: 3079.886962890625


Step: 85 ___________________________________________
val_loss: 4050.27734375


Step: 86 ___________________________________________
val_loss: 4050.291748046875


Step: 87 ___________________________________________
val_loss: 2964.4609375


Step: 88 ___________________________________________
val_loss: 3420.676025390625


Step: 89 ___________________________________________
val_loss: 955.4743041992188


Step: 90 ___________________________________________
val_loss: 4050.294677734375


Step: 91 ___________________________________________
val_loss: 4050.287109375


Step: 92 ___________________________________________
val_loss: 2937.473388671875


Step: 93 ___________________________________________
val_loss: 4213.91650390625


Step: 94 ___________________________________________
val_loss: 4050.952392578125


Step: 95 ___________________________________________
val_loss: 4050.303955078125


Step: 96 ___________________________________________
val_loss: 4050.27

In [18]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 475ms/step - loss: 5826196.0000 - val_loss: 5376470.0000
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 5657958.0000 - val_loss: 7568924.0000
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 6049748.0000 - val_loss: 3504970.7500
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 4293845.5000 - val_loss: 1779083.0000
Epoch 5/10000
1/1 [==============================] - 0s 13ms/step - loss: 3658583.7500 - val_loss: 11982993.0000
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 3948673.0000 - val_loss: 12106783.0000
Epoch 7/10000
1/1 [==============================] - 0s 13ms/step - loss: 4970710.0000 - val_loss: 6968806.5000
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 4668815.0000 - val_loss: 7215700.0000
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 4530549.5000 - val_loss: 63

1/1 [==============================] - 0s 13ms/step - loss: 926865.7500 - val_loss: 1342296.8750
Epoch 74/10000
1/1 [==============================] - 0s 14ms/step - loss: 1113501.3750 - val_loss: 2136479.2500
Epoch 75/10000
1/1 [==============================] - 0s 14ms/step - loss: 1457419.7500 - val_loss: 1042581.0000
Epoch 76/10000
1/1 [==============================] - 0s 13ms/step - loss: 904573.4375 - val_loss: 1115737.0000
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 1370749.6250 - val_loss: 1139201.8750
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 866026.1250 - val_loss: 1000168.3125
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 1131203.1250 - val_loss: 875424.5000
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 999698.5000 - val_loss: 817765.5625
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 842577.6250 - val_loss: 1048207.3125
Epoch

1/1 [==============================] - 0s 13ms/step - loss: 398895.6875 - val_loss: 344725.4688
Epoch 147/10000
1/1 [==============================] - 0s 13ms/step - loss: 468829.2812 - val_loss: 502048.8438
Epoch 148/10000
1/1 [==============================] - 0s 14ms/step - loss: 358242.5938 - val_loss: 267587.0312
Epoch 149/10000
1/1 [==============================] - 0s 13ms/step - loss: 428413.4062 - val_loss: 438578.6562
Epoch 150/10000
1/1 [==============================] - 0s 13ms/step - loss: 379303.5312 - val_loss: 352031.9062
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 458279.2812 - val_loss: 562606.0625
Epoch 152/10000
1/1 [==============================] - 0s 13ms/step - loss: 410591.3438 - val_loss: 596659.5625
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 342697.0625 - val_loss: 309808.7188
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 206912.6250 - val_loss: 222179.1094
Epoch 15

1/1 [==============================] - 0s 14ms/step - loss: 120975.3203 - val_loss: 92306.5234
Epoch 220/10000
1/1 [==============================] - 0s 14ms/step - loss: 161576.3438 - val_loss: 124551.2422
Epoch 221/10000
1/1 [==============================] - 0s 13ms/step - loss: 159362.8750 - val_loss: 69914.7734
Epoch 222/10000
1/1 [==============================] - 0s 13ms/step - loss: 138231.0938 - val_loss: 94383.9375
Epoch 223/10000
1/1 [==============================] - 0s 13ms/step - loss: 165830.6094 - val_loss: 121690.8438
Epoch 224/10000
1/1 [==============================] - 0s 13ms/step - loss: 113125.4531 - val_loss: 125414.0000
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 142821.0469 - val_loss: 74900.6875
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 97133.4375 - val_loss: 77945.0078
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 109636.5625 - val_loss: 90400.8359
Epoch 228/10000

1/1 [==============================] - 0s 14ms/step - loss: 194085.9688 - val_loss: 42656.9297
Epoch 294/10000
1/1 [==============================] - 0s 14ms/step - loss: 68718.0703 - val_loss: 18372.9004
Epoch 295/10000
1/1 [==============================] - 0s 14ms/step - loss: 88193.0859 - val_loss: 7369.7266
Epoch 296/10000
1/1 [==============================] - 0s 13ms/step - loss: 83480.2578 - val_loss: 37355.7852
Epoch 297/10000
1/1 [==============================] - 0s 13ms/step - loss: 43017.8047 - val_loss: 10426.1455
Epoch 298/10000
1/1 [==============================] - 0s 14ms/step - loss: 61885.0586 - val_loss: 14927.4365
Epoch 299/10000
1/1 [==============================] - 0s 13ms/step - loss: 78997.6328 - val_loss: 18199.9297
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 54251.6250 - val_loss: 14611.6357
Epoch 301/10000
1/1 [==============================] - 0s 14ms/step - loss: 147214.4688 - val_loss: 53970.6289
Epoch 302/10000
1/1 [====

...

Epoch 1184/10000
1/1 [==============================] - 0s 14ms/step - loss: 19465.0469 - val_loss: 23486.1816
Epoch 1185/10000
1/1 [==============================] - 0s 14ms/step - loss: 17848.1719 - val_loss: 25288.1777
Epoch 1186/10000
1/1 [==============================] - 0s 13ms/step - loss: 18815.6328 - val_loss: 18823.5352
Epoch 1187/10000
1/1 [==============================] - 0s 14ms/step - loss: 16730.3008 - val_loss: 24327.0156
Epoch 1188/10000
1/1 [==============================] - 0s 13ms/step - loss: 20511.4082 - val_loss: 30266.9043
Epoch 1189/10000
1/1 [==============================] - 0s 14ms/step - loss: 19416.1992 - val_loss: 29364.1016
Epoch 1190/10000
1/1 [==============================] - 0s 13ms/step - loss: 22162.1934 - val_loss: 29929.5840
Epoch 1191/10000
1/1 [==============================] - 0s 13ms/step - loss: 20613.3887 - val_loss: 20417.2500
Epoch 1192/10000
1/1 [==============================] - 0s 13ms/step - loss: 16312.8291 - val_loss: 14053.3984
E

Epoch 1258/10000
1/1 [==============================] - 0s 13ms/step - loss: 26481.8281 - val_loss: 17561.6562
Epoch 1259/10000
1/1 [==============================] - 0s 15ms/step - loss: 19352.3828 - val_loss: 14775.4297
Epoch 1260/10000
1/1 [==============================] - 0s 13ms/step - loss: 15619.9482 - val_loss: 21378.0098
Epoch 1261/10000
1/1 [==============================] - 0s 14ms/step - loss: 20646.1211 - val_loss: 22788.7031
Epoch 1262/10000
1/1 [==============================] - 0s 14ms/step - loss: 17705.6074 - val_loss: 20184.1797
Epoch 1263/10000
1/1 [==============================] - 0s 14ms/step - loss: 44443.3164 - val_loss: 18463.3926
Epoch 1264/10000
1/1 [==============================] - 0s 14ms/step - loss: 12533.8486 - val_loss: 18223.8066
Epoch 1265/10000
1/1 [==============================] - 0s 14ms/step - loss: 19967.4727 - val_loss: 18522.0293
Epoch 1266/10000
1/1 [==============================] - 0s 13ms/step - loss: 28089.2129 - val_loss: 14794.0938
E

Epoch 1332/10000
1/1 [==============================] - 0s 13ms/step - loss: 17357.8711 - val_loss: 16747.0469
Epoch 1333/10000
1/1 [==============================] - 0s 14ms/step - loss: 14643.7646 - val_loss: 19044.2637
Epoch 1334/10000
1/1 [==============================] - 0s 14ms/step - loss: 18909.2207 - val_loss: 19745.0469
Epoch 1335/10000
1/1 [==============================] - 0s 14ms/step - loss: 19412.0664 - val_loss: 24580.1250
Epoch 1336/10000
1/1 [==============================] - 0s 14ms/step - loss: 17687.2441 - val_loss: 23305.5098
Epoch 1337/10000
1/1 [==============================] - 0s 14ms/step - loss: 15534.1914 - val_loss: 23807.1230
Epoch 1338/10000
1/1 [==============================] - 0s 13ms/step - loss: 16760.3594 - val_loss: 23952.5918
Epoch 1339/10000
1/1 [==============================] - 0s 14ms/step - loss: 16536.4141 - val_loss: 31058.4531
Epoch 1340/10000
1/1 [==============================] - 0s 13ms/step - loss: 19725.8359 - val_loss: 21602.0723
E

Epoch 1406/10000
1/1 [==============================] - 0s 13ms/step - loss: 11762.8398 - val_loss: 17116.3926
Epoch 1407/10000
1/1 [==============================] - 0s 13ms/step - loss: 13880.4336 - val_loss: 15071.1279
Epoch 1408/10000
1/1 [==============================] - 0s 13ms/step - loss: 14645.4395 - val_loss: 14929.5820
Epoch 1409/10000
1/1 [==============================] - 0s 13ms/step - loss: 14654.3535 - val_loss: 13711.9854
Epoch 1410/10000
1/1 [==============================] - 0s 13ms/step - loss: 20916.6328 - val_loss: 28440.1973
Epoch 1411/10000
1/1 [==============================] - 0s 13ms/step - loss: 32014.8164 - val_loss: 34450.1914
Epoch 1412/10000
1/1 [==============================] - 0s 14ms/step - loss: 25883.9785 - val_loss: 26561.4160
Epoch 1413/10000
1/1 [==============================] - 0s 14ms/step - loss: 17632.7910 - val_loss: 14691.9268
Epoch 1414/10000
1/1 [==============================] - 0s 14ms/step - loss: 11235.7568 - val_loss: 22125.1406
E

Epoch 1480/10000
1/1 [==============================] - 0s 14ms/step - loss: 11793.0947 - val_loss: 13937.9756
Epoch 1481/10000
1/1 [==============================] - 0s 13ms/step - loss: 12909.2773 - val_loss: 17547.5469
Epoch 1482/10000
1/1 [==============================] - 0s 13ms/step - loss: 11695.9229 - val_loss: 13216.3477
Epoch 1483/10000
1/1 [==============================] - 0s 14ms/step - loss: 30995.7949 - val_loss: 22829.8125
Epoch 1484/10000
1/1 [==============================] - 0s 13ms/step - loss: 25820.8242 - val_loss: 20904.7656
Epoch 1485/10000
1/1 [==============================] - 0s 13ms/step - loss: 14780.9023 - val_loss: 12829.8789
Epoch 1486/10000
1/1 [==============================] - 0s 13ms/step - loss: 18887.4355 - val_loss: 15974.2256
Epoch 1487/10000
1/1 [==============================] - 0s 13ms/step - loss: 20018.4941 - val_loss: 12423.7383
Epoch 1488/10000
1/1 [==============================] - 0s 14ms/step - loss: 31111.0156 - val_loss: 18074.1035
E

In [19]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[2782.135](test_target) - [[2375.6577]](prediction) = 406.4772851562502


In [20]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [21]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Goiás - Desemprego,Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiás - Produção de Cimento (t),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado
0,1.832928,-1.708338,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-1.119467,...,1.274170,1.665189,-1.012620,-1.137668,-0.496136,1.924728,-1.556881,-1.701956,-1.711402,-1.698591
1,1.025450,-1.108653,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-0.924403,...,1.142436,0.336548,0.693805,0.744267,0.049688,1.168297,-1.061470,-1.044093,-1.150571,-1.080656
2,0.749626,-0.565930,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.799382,...,1.039633,-1.054680,1.889424,1.962566,-0.308728,-0.221154,-0.603594,-0.613035,-0.653693,-0.576048
3,-0.262764,-0.079236,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.601421,...,0.206309,-1.346842,0.953086,0.438670,-1.680433,-0.697199,-0.154604,-0.088148,0.063250,-0.001415
4,-0.624935,0.352365,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,-0.058811,...,-1.114366,-0.945435,-0.283728,-0.559814,-0.815864,-0.886708,0.246543,0.418077,0.599093,0.434837
5,-0.984636,0.729808,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,0.553626,...,-1.375681,-0.176848,-0.551951,0.138980,1.587873,-1.155445,0.574937,0.603147,0.794207,0.635208
6,-0.653921,1.054027,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,1.135251,...,-0.837490,0.532911,-0.523784,-0.285717,1.193362,-0.430555,0.827173,0.940694,0.708371,0.691586
7,-1.081748,1.325957,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,1.814607,...,-0.335011,0.989157,-1.164233,-1.301284,0.470238,0.298037,1.727896,1.485313,1.350743,1.595079


train_target:


,Goiás - Consumo de Cimento (t)
0,1514.939
1,1716.900
2,1730.142
3,1943.769
4,2310.616
5,2335.717
6,2657.527
7,2901.707


test_input:


...

,Goiás - Desemprego,Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiás - Produção de Cimento (t),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado
17,0.596674,-0.104074,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.913043,...,0.207996,0.427875,0.763149,-1.965098,-1.88718,-2.218598,0.832746,-0.850806,-0.449768,0.60687


test_target:


,Goiás - Consumo de Cimento (t)
17,2782.135


1/1 [==============================] - 0s 35ms/step
Error: 406.4772851562502




[2662.422607421875,
 2874.562255859375,
 3062.04248046875,
 2758.6728515625,
 2740.908447265625,
 2443.373779296875,
 2236.72607421875,
 2229.74267578125,
 2306.65869140625,
 2375.65771484375]

In [22]:
display(targets)
display(predictions)

[3061.533,
 3181.778,
 2861.8974285714285,
 2517.2907428571425,
 2193.5458285714285,
 2270.346,
 2282.903,
 2367.716,
 2438.585,
 2782.135]

[2662.422607421875,
 2874.562255859375,
 3062.04248046875,
 2758.6728515625,
 2740.908447265625,
 2443.373779296875,
 2236.72607421875,
 2229.74267578125,
 2306.65869140625,
 2375.65771484375]

In [23]:
mae = mean_absolute_error(predictions, targets)
mae

259.07975390625

In [24]:
porcentage = mae/np.mean(targets)
porcentage

0.09980832449765446